In [ ]:
'''Comparison'''

import pandas as pd
import numpy as np
import sys, os, pathlib
import joblib 
import time 


NOTEBOOK_DIR = pathlib.Path(os.getcwd())  
ROOT = NOTEBOOK_DIR.parents[0]  

sys.path.insert(0, str(ROOT))

from src.dataset_generator import sample_heston_lhs
from src.Heston_FFT import heston_price_fft
from src.Heston_MC import heston_mc_qe_price
heston_nn = joblib.load("..\src\model_ann_3.pkl")


# FFT params
alpha  = 1.5
fft_N  = 4096     
eta    = 0.1      

# MC params
mc_steps = 150
mc_paths = 100_000
seed     = 42

# Random dataset (LHS)
test_df = sample_heston_lhs(100, seed=123, S0=105, q=0)
test_df = pd.DataFrame(test_df)


# Solver
def solver(df, alpha, eta, N, steps, path):

    S       = df['S0']
    K       = df['K']
    T       = df['T']
    r       = df['r']
    v0      = df['v0']
    kappa   = df['kappa']
    theta   = df['theta']
    sigma_v = df['sigma_v']
    rho     = df['rho']
    q       = df['q']

    # results
    fft_price, fft_time = [], []
    mc_price,  mc_time  = [], []
    nn_price,  nn_time  = [], []

    # loop 
    for i, row in df.iterrows():
        Si, Ki, Ti = float(row['S0']), float(row['K']), float(row['T'])
        ri, v0i    = float(row['r']), float(row['v0'])
        kappai, thetai = float(row['kappa']), float(row['theta'])
        sigi, rhoi = float(row['sigma_v']), float(row['rho'])
        qi = float(row['q'])

        # 1- FFT
        t0 = time.perf_counter()
        p_fft = heston_price_fft(
            S=Si, K=Ki, T=Ti, r=ri, v0=v0i,
            kappa=kappai, theta=thetai, sigma_v=sigi, rho=rhoi,
            option_type="call", q=qi, alpha=alpha, N=N, eta=eta
        )
        t1 = time.perf_counter()

        # 2- MC 
        t2 = time.perf_counter()
        p_mc = heston_mc_qe_price(
            Si, Ki, Ti, ri, qi,
            v0i, kappai, thetai, sigi, rhoi,
            steps=steps, paths=path,
            antithetic=True, seed=seed+i,
            option_type="call"
        )
        t3 = time.perf_counter()
        if isinstance(p_mc, tuple):
            p_mc = p_mc[0]


        # push risultati
        fft_price.append(float(p_fft))
        fft_time.append(t1 - t0)

        mc_price.append(float(p_mc))
        mc_time.append(t3 - t2)
    
    # 3- NN
    feats = ["m", "T", "r", "v0", "kappa", "theta", "sigma_v", "rho"]
    X = df[feats].to_numpy(dtype=float)
    t4 = time.perf_counter()
    p_nn = heston_nn.predict(X).astype(float).ravel()
    t5 = time.perf_counter()


    # cols
    df['FFT_price'] = fft_price
    df['FFT_time']  = fft_time

    df['MC_price']  = mc_price      
    df['MC_time']   = mc_time
    df['MC_error']  = abs(df['MC_price'] - df['FFT_price'])
    df['MC_error_rel']  = (df['MC_error']/np.maximum(np.abs(df['FFT_price']), 1e-6))
    df["MC_error_%"] = 100 * df["MC_error_rel"]

    df['NN_price'] = p_nn
    df['NN_time']  = (t5 - t4) / len(df) # already the mean
    df['NN_error']  = abs(df['NN_price'] - df['FFT_price'])
    df['NN_error_rel']  = (df['NN_error']/np.maximum(np.abs(df['FFT_price']), 1e-6))
    df["NN_error_%"] = 100 * df["NN_error_rel"]

    df.drop(columns=['MC_error_rel', 'NN_error_rel'], inplace=True)

    return df


results_df = solver(test_df, alpha, eta, fft_N, mc_steps, mc_paths)
#results_df.to_csv('data/results_df.csv', index=False)
results_df

c:\Users\giovanni.piva\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,S0,K,m,T,r,q,v0,kappa,theta,sigma_v,...,FFT_price,FFT_time,MC_price,MC_time,MC_error,MC_error_%,NN_price,NN_time,NN_error,NN_error_%
0,105,115.158920,1.096752,0.811002,0.092148,0,0.076527,4.841289,0.071912,0.418670,...,9.209616,0.001082,9.289280,0.854499,0.079664,0.865011,9.222542,0.000667,0.012925,0.140347
1,105,117.510220,1.119145,0.969393,0.033654,0,0.014872,1.874977,0.087795,0.283398,...,5.532732,0.001297,5.561658,0.859189,0.028925,0.522803,5.638286,0.000667,0.105554,1.907802
2,105,100.730320,0.959336,1.141289,0.032030,0,0.059229,4.702282,0.064080,0.354168,...,15.253949,0.001098,15.243425,0.801079,0.010523,0.068987,14.914779,0.000667,0.339170,2.223487
3,105,105.577276,1.005498,0.535943,0.073124,0,0.038726,2.389665,0.085412,0.168723,...,9.194507,0.000883,9.198795,0.755147,0.004289,0.046647,8.909756,0.000667,0.284750,3.096958
4,105,135.499952,1.290476,1.206278,0.050265,0,0.059001,4.854463,0.055949,0.413841,...,3.819320,0.001072,3.772125,1.120528,0.047195,1.235682,3.917590,0.000667,0.098270,2.572974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,105,123.320228,1.174478,0.218166,0.005058,0,0.021303,3.953465,0.078413,0.724006,...,0.002820,0.001196,0.002468,0.722282,0.000353,12.504676,0.260496,0.000667,0.257676,9136.521213
96,105,115.846338,1.103298,0.142529,0.010206,0,0.030798,4.693649,0.011166,0.257579,...,0.152711,0.000925,0.147451,0.695058,0.005260,3.444352,0.735906,0.000667,0.583195,381.893427
97,105,105.095538,1.000910,0.550792,-0.007123,0,0.026378,4.117916,0.063895,0.453379,...,6.322990,0.001134,6.336913,0.707679,0.013923,0.220196,6.152672,0.000667,0.170318,2.693633
98,105,97.119062,0.924943,1.159459,0.035700,0,0.016154,4.573358,0.020382,0.285945,...,13.817800,0.000953,13.849992,0.686851,0.032192,0.232974,14.129825,0.000667,0.312025,2.258139


In [34]:
# check

fft_try =  heston_price_fft(S=105, K=115.159, T=0.811, r=0.0921, v0=0.0765, kappa=4.841, theta=0.072, sigma_v=0.419, rho=-0.177,
                             option_type="call", q=0, alpha=alpha, N=fft_N, eta=eta)
MC_try =  heston_mc_qe_price(S0=105,  K=115.159, T=0.811, r=0.0921, q=0, v0=0.0765, kappa=4.841, theta=0.072, sigma_v=0.419, rho=-0.177,
                                         steps=mc_steps, paths=mc_paths,
                                         antithetic=True, seed=seed, option_type="call")
df_NN_try = pd.DataFrame([{"m": 1.0967,"T": 0.81,"r": 0.092,"v0": 0.076,"kappa": 4.841,"theta": 0.072,"sigma_v": 0.419,"rho": -0.177}])
NN_try = heston_nn.predict(df_NN_try)[0]

print(f"fft: {fft_try:.3f}")
print(f"mc: {MC_try[0]:.3f}")
print(f"nn: {NN_try:.3f}")

fft: 9.212
mc: 9.239
nn: 9.203


In [6]:
fft_avg_time = results_df['FFT_time'].mean()
mc_avg_time = results_df['MC_time'].mean()
nn_avg_time = results_df['NN_time'].mean()

fft_std_time = results_df['FFT_time'].std()
mc_std_time  = results_df['MC_time'].std()
nn_std_time  = results_df['NN_time'].std()

fft_total_time = results_df['FFT_time'].sum()
mc_total_time  = results_df['MC_time'].sum()
nn_total_time  = results_df['NN_time'].sum()

mc_avg_error = results_df['MC_error'].mean().round(7)
nn_avg_error = results_df['NN_error'].mean().round(7)

mc_avg_error_perc = results_df['MC_error_%'].mean().round(3)
nn_avg_error_perc = results_df['NN_error_%'].mean().round(3) # strongly influenced by outliers!

mc_med_error_perc = results_df['MC_error_%'].median().round(3)
nn_med_error_perc = results_df['NN_error_%'].median().round(3) 

mc_std_error = results_df['MC_error'].std().round(3)
nn_std_error = results_df['NN_error'].std().round(3)

mc_max_error_perc = results_df['MC_error_%'].max().round(3)
nn_max_error_perc = results_df['NN_error_%'].max().round(3)


summary = pd.DataFrame({
    "avg_time":   [fft_avg_time, mc_avg_time, nn_avg_time],
    "std_time":   [fft_std_time, mc_std_time, nn_std_time],
    "avg_error":  [0, mc_avg_error, nn_avg_error],   # FFT baseline = 0
    "avg_rel_error_(%)":  [0, mc_avg_error_perc, nn_avg_error_perc],
    "med_rel_error_(%)":  [0, mc_med_error_perc, nn_med_error_perc],
    "std_error":  [0, mc_std_error, nn_std_error],
    "max_rel_error_(%)":  [0, mc_max_error_perc, nn_max_error_perc]
}, index=["FFT", "MC", "NN"])

summary

,avg_time,std_time,avg_error,avg_rel_error_(%),med_rel_error_(%),std_error,max_rel_error_(%)
FFT,0.001068,1.758238e-04,0.000000,0.000,0.000,0.000,0.000000e+00
MC,0.753434,1.128610e-01,0.031859,3.891,0.243,0.051,1.616520e+02
NN,0.000667,4.358657e-19,0.236186,699700.347,1.632,0.182,6.825335e+07
